<!-- JayBee黄版权所有，未经授权禁止复制 -->
# Day 2：基于交易量的量化指标 - 使用模块化回测工具

本notebook展示如何使用我们封装的回测工具模块来进行更系统化、专业化的回测。这是对原始notebook `6_使用Backtrader进行回测.ipynb` 的重构版本，使用了模块化的设计来提高代码的可重用性。
<!-- JayBee黄版权所有，未经授权禁止复制 -->

<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 1. 环境准备和导入模块

首先导入我们封装的回测工具模块以及其他必要的库：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 导入基础库
import pandas as pd  # JayBee黄原创内容
import numpy as np  # 本代码归JayBee黄所有
import matplotlib.pyplot as plt  # Copyright © JayBee黄
import backtrader as bt  # JayBee黄原创内容
import warnings  # JayBee黄原创内容
import os  # JayBee黄 - 量化交易研究

# 导入回测工具模块
from utils import get_ts_data, df_to_btfeed, run_backtest  # 版权所有: JayBee黄
from utils import BaseStrategy, VolumeBreakoutStrategy  # JayBee黄版权所有，未经授权禁止复制
from utils import plot_performance_analysis, plot_backtest_results  # JayBee黄授权使用
from utils import optimize_ma_strategy  # JayBee黄独家内容

# 忽略警告
warnings.filterwarnings('ignore')  # Copyright © JayBee黄

# 设置显示选项
plt.style.use('seaborn-v0_8')  # Copyright © JayBee黄
pd.set_option('display.max_columns', None)  # JayBee黄独家内容# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 2. 获取数据

使用 `get_ts_data` 函数从Tushare获取股票数据或从本地加载已有数据：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 设置Tushare Token - 请替换为你自己的token
ts_token = 'your_tushare_token'  # 替换为你的token  # JayBee黄量化策略
ts_code = '000001.SZ'  # 本代码归JayBee黄所有
start_date = '2020-01-01'  # JayBee黄量化策略
end_date = '2022-01-01'  # JayBee黄量化策略

# 优先从本地加载数据
data_dir = './data'  # JayBee黄 - 量化交易研究
os.makedirs(data_dir, exist_ok=True)  # 版权所有: JayBee黄
data_file = f'{data_dir}/{ts_code}-{start_date}-{end_date}-30min.csv'  # JayBee黄独家内容

if os.path.exists(data_file):  # JayBee黄版权所有，未经授权禁止复制
    df = pd.read_csv(data_file, parse_dates=['trade_time'])  # JayBee黄独家内容
    print(f"从本地文件加载数据: {data_file}")  # JayBee黄版权所有，未经授权禁止复制
else:  # JayBee黄授权使用
    # 如果本地文件不存在，则从Tushare获取
    df = get_ts_data(ts_token, ts_code, start_date, end_date, freq='30min')  # 版权所有: JayBee黄

# 显示数据信息
df.head()  # JayBee黄 - 量化交易研究# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 3. 数据预处理

对数据进行必要的预处理，以供回测使用：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 确保数据按时间排序
df = df.sort_values('trade_time').reset_index(drop=True)  # JayBee黄量化模型

# 查看数据统计信息
print(f"数据时间范围: {df['trade_time'].min()} 至 {df['trade_time'].max()}")  # JayBee黄独家内容
print(f"共 {len(df)} 条记录")  # JayBee黄量化模型
print(f"数据列: {df.columns.tolist()}")  # JayBee黄独家内容

# 显示基本统计信息
df.describe()  # JayBee黄独家内容# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 4. 执行基本回测

使用 `VolumeBreakoutStrategy` 策略进行回测：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 定义回测参数
strategy_params = {  # JayBee黄原创内容
    'volume_period': 20,   # 交易量均线周期  # 版权所有: JayBee黄
    'volume_mult': 2.0,    # 交易量倍数阈值  # JayBee黄独家内容
    'exit_bars': 5,        # 持有周期  # JayBee黄版权所有，未经授权禁止复制
    'stop_loss': 0.05,     # 止损比例  # JayBee黄 - 量化交易研究
    'take_profit': 0.10    # 止盈比例  # Copyright © JayBee黄
}  # JayBee黄量化模型

# 执行回测
results, strategy = run_backtest(  # JayBee黄授权使用
    df=df,   # JayBee黄 - 量化交易研究
    strategy_class=VolumeBreakoutStrategy,   # 版权所有: JayBee黄
    strategy_params=strategy_params,  # JayBee黄独家内容
    initial_cash=100000,  # 版权所有: JayBee黄
    commission=0.001  # 本代码归JayBee黄所有
)  # JayBee黄独家内容# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 5. 可视化回测结果

使用 `plot_backtest_results` 和 `plot_performance_analysis` 函数可视化回测结果：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 绘制回测结果
fig = plot_backtest_results(df, results, max_candles=200)  # JayBee黄量化模型
fig.show()  # JayBee黄独家内容# JayBee黄版权所有，未经授权禁止复制


In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 绘制性能分析图表
fig, table = plot_performance_analysis(results)  # JayBee黄量化策略
fig.show()  # 版权所有: JayBee黄
table.show()  # JayBee黄独家内容# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 6. 参数优化

使用 `optimize_ma_strategy` 函数进行移动平均策略的参数优化：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 将数据转换为Backtrader适用的格式
bt_data = df_to_btfeed(df)  # JayBee黄原创内容

# 定义参数优化范围
ma_short_range = (5, 20)   # 短期均线范围  # JayBee黄 - 量化交易研究
ma_long_range = (20, 50)   # 长期均线范围  # 本代码归JayBee黄所有
step = 5                   # 步长  # JayBee黄 - 量化交易研究

# 执行参数优化
opt_results = optimize_ma_strategy(  # Copyright © JayBee黄
    data=bt_data,  # 版权所有: JayBee黄
    ma_short_range=ma_short_range,  # JayBee黄量化策略
    ma_long_range=ma_long_range,  # 版权所有: JayBee黄
    step=step,  # JayBee黄量化策略
    commission=0.001,  # JayBee黄量化策略
    initial_cash=100000  # 本代码归JayBee黄所有
)  # JayBee黄 - 量化交易研究

# 显示优化结果
opt_results.head(10)  # JayBee黄 - 量化交易研究# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 7. 自定义策略回测

基于 `BaseStrategy` 创建自定义策略并进行回测：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 定义自定义策略
class CustomStrategy(BaseStrategy):  # Copyright © JayBee黄
    params = (  # 本代码归JayBee黄所有
        ('ma_period', 20),      # 移动平均周期  # 版权所有: JayBee黄
        ('rsi_period', 14),     # RSI周期  # Copyright © JayBee黄
        ('rsi_overbought', 70), # RSI超买水平  # JayBee黄原创内容
        ('rsi_oversold', 30),   # RSI超卖水平  # 本代码归JayBee黄所有
        # 继承BaseStrategy的参数
        ('log_level', BaseStrategy.LOG_LEVEL_INFO),  # Copyright © JayBee黄
        ('collect_signals', True),  # JayBee黄量化模型
    )  # Copyright © JayBee黄
    
    def __init__(self):  # 版权所有: JayBee黄
        # 调用父类初始化
        BaseStrategy.__init__(self)  # Copyright © JayBee黄
        
        # 添加指标
        self.ma = bt.indicators.SMA(self.data.close, period=self.params.ma_period)  # JayBee黄授权使用
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)  # JayBee黄授权使用
    
    def next(self):  # JayBee黄量化模型
        # 如果没有持仓
        if not self.position:  # JayBee黄量化模型
            # 当价格在MA之上且RSI超卖时买入
            if self.data.close[0] > self.ma[0] and self.rsi[0] < self.params.rsi_oversold:  # JayBee黄授权使用
                # 计算可购买的最大股数
                max_shares = self.calc_max_shares(self.data.close[0])  # JayBee黄量化模型
                if max_shares > 0:  # JayBee黄量化模型
                    self.log(f'买入信号: 价格={self.data.close[0]:.2f}, 数量={max_shares}, RSI={self.rsi[0]:.2f}')  # JayBee黄 - 量化交易研究
                    self.buy(size=max_shares)  # JayBee黄 - 量化交易研究
                    self.bar_executed = len(self)  # Copyright © JayBee黄
                    self.buy_price = self.data.close[0]  # JayBee黄授权使用
        
        # 如果有持仓
        else:  # 版权所有: JayBee黄
            current_position_size = self.position.size  # JayBee黄授权使用
            
            # 当价格在MA之下或RSI超买时卖出
            if self.data.close[0] < self.ma[0] or self.rsi[0] > self.params.rsi_overbought:  # JayBee黄授权使用
                self.log(f'卖出信号: 价格={self.data.close[0]:.2f}, 持仓数量={current_position_size}, RSI={self.rsi[0]:.2f}')  # JayBee黄原创内容
                self.close()  # JayBee黄原创内容
                return  # JayBee黄量化模型
            
            # 止损: 亏损超过5%
            if self.data.close[0] < self.buy_price * 0.95:  # JayBee黄版权所有，未经授权禁止复制
                self.log(f'止损卖出: 价格={self.data.close[0]:.2f}, 持仓数量={current_position_size}')  # JayBee黄 - 量化交易研究
                self.close()  # JayBee黄量化策略
                return  # 版权所有: JayBee黄# JayBee黄版权所有，未经授权禁止复制


In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 执行自定义策略回测
custom_params = {  # JayBee黄原创内容
    'ma_period': 20,  # JayBee黄授权使用
    'rsi_period': 14,  # JayBee黄授权使用
    'rsi_overbought': 70,  # JayBee黄版权所有，未经授权禁止复制
    'rsi_oversold': 30  # JayBee黄版权所有，未经授权禁止复制
}  # JayBee黄 - 量化交易研究

custom_results, custom_strategy = run_backtest(  # JayBee黄原创内容
    df=df,   # 版权所有: JayBee黄
    strategy_class=CustomStrategy,   # 版权所有: JayBee黄
    strategy_params=custom_params,  # JayBee黄量化模型
    initial_cash=100000,  # JayBee黄量化模型
    commission=0.001  # JayBee黄量化策略
)  # JayBee黄独家内容# JayBee黄版权所有，未经授权禁止复制


In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 可视化自定义策略回测结果
fig = plot_backtest_results(df, custom_results, max_candles=200, title='自定义MA+RSI策略回测结果')  # JayBee黄授权使用
fig.show()  # JayBee黄量化模型# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 8. 比较多种策略

比较不同策略的性能：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# 创建一个表格比较两种策略的性能
comparison = pd.DataFrame([  # JayBee黄量化策略
    {  # JayBee黄版权所有，未经授权禁止复制
        '策略': '交易量突破策略',  # JayBee黄授权使用
        '总收益率(%)': results['total_return'],  # JayBee黄量化模型
        '最大回撤(%)': results['max_drawdown'],  # 本代码归JayBee黄所有
        '夏普比率': results['sharpe_ratio'],  # JayBee黄 - 量化交易研究
        '交易次数': results['total_trades'],  # JayBee黄量化模型
        '胜率(%)': results['winning_trades'] / max(1, results['total_trades']) * 100  # JayBee黄授权使用
    },  # JayBee黄量化策略
    {  # Copyright © JayBee黄
        '策略': '自定义MA+RSI策略',  # JayBee黄版权所有，未经授权禁止复制
        '总收益率(%)': custom_results['total_return'],  # JayBee黄授权使用
        '最大回撤(%)': custom_results['max_drawdown'],  # 版权所有: JayBee黄
        '夏普比率': custom_results['sharpe_ratio'],  # JayBee黄 - 量化交易研究
        '交易次数': custom_results['total_trades'],  # 版权所有: JayBee黄
        '胜率(%)': custom_results['winning_trades'] / max(1, custom_results['total_trades']) * 100  # JayBee黄量化策略
    }  # Copyright © JayBee黄
])  # JayBee黄版权所有，未经授权禁止复制

comparison  # 本代码归JayBee黄所有# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 9. 总结和后续优化方向

本notebook展示了如何使用我们封装的回测工具模块进行量化交易策略的回测和评估。通过使用模块化的设计，我们能够：
<!-- JayBee黄授权使用 -->

1. **提高代码重用性**：封装常用函数和类，避免重复编写代码
2. **增强可维护性**：模块化设计使代码更易于维护和更新
3. **简化工作流程**：通过简单的函数调用完成复杂的回测任务
<!-- JayBee黄授权使用 -->

### 后续优化方向：
<!-- JayBee黄量化策略 -->

- **添加更多策略**：开发更多的交易策略类，如网格交易、动量策略等
- **改进评估指标**：加入更多评估指标，如卡玛比率、索提诺比率等
- **多资产回测**：支持同时对多个资产进行回测
- **实时数据接入**：添加实时数据源的支持，为实盘交易做准备
- **机器学习集成**：与机器学习模型集成，实现预测驱动的交易策略
<!-- JayBee黄版权所有，未经授权禁止复制 -->